In [13]:
import os
import re
import requests
import sys
import os
import pandas as pd
import numpy as np
import tiktoken
import os
# importing necessary functions from dotenv library
from dotenv import load_dotenv, dotenv_values
import faiss
import numpy as np
import pymysql
import boto3
import pickle


 
bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-east-1', 
)

In [14]:
# load_dotenv(".env")


# endpoint = os.getenv('RDS_ENDPOINT')
# username = os.getenv('RDS_USERNAME')
# password = os.getenv('RDS_PASSWORD')
# database = os.getenv('RDS_DATABASE')

port = 3306

endpoint="database-colmobil.c9owiq2sebpi.us-east-1.rds.amazonaws.com"
username="admin"
password="Bb123456!"
database="databasecolmobil"
# Establish a connection to the RDS instance
connection = pymysql.connect(
    host=endpoint,
    user=username,
    password=password,
    database=database
)
cursor = connection.cursor()
# Execute a query
cursor.execute("SELECT VERSION()")
version = cursor.fetchone()
print(f"Database version: {version}")

Database version: ('8.0.39',)


In [15]:
select_sql = "SELECT car_id, additional_description, model FROM cars_collection;"
        
        # Execute the query\
cursor.execute(select_sql)

# Fetch all results
results = cursor.fetchall()
print(results)
# Create a DataFrame from the results
df_cars = pd.DataFrame(results, columns=['car_id', 'additional_description', 'model'])

((1, 'רכב שטח אידיאלי למשפחה שמחפשת את השילוב המושלם בין נוחות, עיצוב מודרני וביצועים גבוהים. עם מרווח פנימי שמספק נוחות לכל הנוסעים ויכולת תמרון מצוינת בעיר ובשטח קל, הטוסון מתאים גם לנסיעות יומיומיות וגם להרפתקאות בסוף השבוע. יתרון בולט: שילוב מושלם של ביצועים, נוחות ועיצוב שמתאים לכל דרך', 'Tucson'), (2, 'רכב שטח פרימיום שמפגיש בין עיצוב מרהיב לנוחות גבוהה, מושלם למשפחות שמחפשות מקום נוסף לנסיעות ארוכות וטיולים משותפים. עם מערכת 4x4 מצוינת ויכולת להתמודד עם שטח קל, הוא מציע חוויית נהיגה נוחה ומרשימה. יתרון בולט: מערכת היברידית שמספקת ביצועים מרשימים לצד חיסכון בדלק', 'Santa Fe'), (3, 'רכב שטח קומפקטי שמציע עיצוב צעיר ודינמי, אידיאלי לנהיגה בעיר ולטיולים קלים בשטח. עם מערכת תמרון מצוינת, הוא מספק חוויית נהיגה מהנה וקלילה. יתרון בולט: עיצוב מודרני שמפנה מקום לרכב שטח קטן אך חזק', 'Kona'), (4, 'רכב חשמלי מתקדם עם עיצוב עתידני וטכנולוגיה חדשנית. אידיאלי לאורח חיים עירוני, הוא מספק ביצועים גבוהים וחוויית נהיגה שקטה וירוקה. יתרון בולט: חוויית נהיגה שקטה, מבלי להתפשר על ביצועים', ' Ioniq 5

In [16]:
import boto3
import json
 
#Create the connection to Bedrock


# Define prompt and model parameters
modelId = "amazon.titan-embed-text-v2:0"   

def generate_embeddings(prompt_data, modelId = "amazon.titan-embed-text-v2:0"):
    accept = "application/json"
    contentType = "application/json"
    sample_model_input={
        "inputText": prompt_data,
        "dimensions": 256,
        "normalize": True
    }

    body = json.dumps(sample_model_input)
    # Invoke model
    response = bedrock_runtime.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)

    response_body = json.loads(response.get('body').read())
    embedding = response_body.get("embedding")
    # Print response and embedding
    print(f"The embedding vector has {len(embedding)} values\n{embedding[0:3]+['...']+embedding[-3:]}")
    return np.array(embedding)

# Create the combined_info column
df_recipes['combined_info'] = df_recipes.apply(
    lambda row: row['recipe_name'] + ' - ' + row['instructions_and_ingredients'],
    axis=1
)


In [17]:
df_cars['embeddings_vector_additional_description'] = df_cars['additional_description'].apply(lambda x : generate_embeddings (x))
df_cars['embeddings_vector_model'] = df_cars["model"].apply(lambda x : generate_embeddings (x))

The embedding vector has 256 values
[-0.04292425513267517, 0.024676263332366943, 0.04541261866688728, '...', -0.02125476486980915, 0.03380025923252106, 0.008139019832015038]
The embedding vector has 256 values
[-0.06886561959981918, 0.030542096123099327, 0.06964375823736191, '...', -0.017605476081371307, 0.02062077820301056, 0.004984975792467594]
The embedding vector has 256 values
[-0.04771381989121437, -0.025483516976237297, 0.029821136966347694, '...', -0.015362403355538845, 0.0379541739821434, 0.005647942423820496]
The embedding vector has 256 values
[-0.05778118595480919, 0.11071844398975372, 0.027333615347743034, '...', -0.02404666133224964, -0.011331340298056602, 0.007395646069198847]
The embedding vector has 256 values
[-0.12002024054527283, 0.07033033668994904, 0.019111502915620804, '...', -0.057716742157936096, 0.046249840408563614, 0.009077964350581169]
The embedding vector has 256 values
[-0.04647618159651756, 0.024978388100862503, 0.07984894514083862, '...', -0.01054415572

In [18]:
df_cars['embeddings_vector_additional_description']

0     [-0.04292425513267517, 0.024676263332366943, 0...
1     [-0.06886561959981918, 0.030542096123099327, 0...
2     [-0.04771381989121437, -0.025483516976237297, ...
3     [-0.05778118595480919, 0.11071844398975372, 0....
4     [-0.12002024054527283, 0.07033033668994904, 0....
5     [-0.04647618159651756, 0.024978388100862503, 0...
6     [0.002815612591803074, 0.11627718806266785, 0....
7     [-0.10661908984184265, 0.028212517499923706, 0...
8     [-0.08635088801383972, 0.015401766635477543, -...
9     [-0.03301191329956055, 0.004370660055428743, -...
10    [-0.05944891646504402, 0.12119907885789871, -0...
11    [-0.10888701677322388, 0.09688372910022736, -0...
12    [-0.06422393023967743, 0.05481971055269241, 0....
13    [-0.09265321493148804, -0.008034119382500648, ...
14    [-0.1762755960226059, -0.034944500774145126, -...
15    [-0.06025490537285805, 0.09480667114257812, 0....
16    [-0.11116329580545425, 0.07804117351770401, -0...
17    [-0.1464223861694336, 0.06627539545297623,

In [19]:
def saving_embedding_data_cars(id,embeddings_vector_additional_description, embeddings_vector_model):
        
        # Convert the embedding vector to bytes
        embeddings_vector_additional_description_bytes = pickle.dumps(embeddings_vector_additional_description)
        embedding_vector_model_bytes = pickle.dumps(embeddings_vector_model)
        
        # Define the SQL update statement
        update_sql = f"""
        UPDATE cars_collection
        SET embeddings_vector_additional_description = %s,
        embeddings_vector_model = %s        
        WHERE car_id = %s
        """
        
        # Execute the update statement (replace `1` with the appropriate row ID)
        cursor.execute(update_sql, (embeddings_vector_additional_description_bytes,embedding_vector_model_bytes, id))
        
        # Commit the changes
        connection.commit()

In [20]:
for id in df_cars['car_id']:
    embeddings_vector_additional_description = df_cars.loc[df_cars['car_id'] == id].embeddings_vector_additional_description
    embedding_vector_model = df_cars.loc[df_cars['car_id'] == id].embeddings_vector_model
    saving_embedding_data_cars(id,embeddings_vector_additional_description, embedding_vector_model)

In [ ]:
select_sql = f"SELECT car_id, embeddings_vector_additional_description, embeddings_vector_model FROM cars_collection;"

# Execute the query
cursor.execute(select_sql)

# Fetch all results
results = cursor.fetchall()

In [23]:
results

((1,
  b'\x80\x04\x95\xf4\n\x00\x00\x00\x00\x00\x00\x8c\x12pandas.core.series\x94\x8c\x06Series\x94\x93\x94)\x81\x94}\x94(\x8c\x04_mgr\x94\x8c\x1epandas.core.internals.managers\x94\x8c\x12SingleBlockManager\x94\x93\x94)\x81\x94(]\x94\x8c\x18pandas.core.indexes.base\x94\x8c\n_new_Index\x94\x93\x94h\x0b\x8c\x05Index\x94\x93\x94}\x94(\x8c\x04data\x94\x8c\x15numpy.core.multiarray\x94\x8c\x0c_reconstruct\x94\x93\x94\x8c\x05numpy\x94\x8c\x07ndarray\x94\x93\x94K\x00\x85\x94C\x01b\x94\x87\x94R\x94(K\x01K\x01\x85\x94h\x15\x8c\x05dtype\x94\x93\x94\x8c\x02i8\x94\x89\x88\x87\x94R\x94(K\x03\x8c\x01<\x94NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00t\x94b\x89C\x08\x00\x00\x00\x00\x00\x00\x00\x00\x94t\x94b\x8c\x04name\x94Nu\x86\x94R\x94a]\x94h\x14h\x17K\x00\x85\x94h\x19\x87\x94R\x94(K\x01K\x01\x85\x94h\x1e\x8c\x02O8\x94\x89\x88\x87\x94R\x94(K\x03\x8c\x01|\x94NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK?t\x94b\x89]\x94h\x14h\x17K\x00\x85\x94h\x19\x87\x94R\x94(K\x01M\x00\x01\x85\x94h\x1e\x8c\x02f8\x94\x89\x88\x8

In [24]:

# Commit the changes
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()